In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

import numpy as np


# Create a Spark session
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()
sc = SparkContext.getOrCreate()
spark.sparkContext.setLogLevel("ERROR")



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/09 15:38:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

def readFile(filename):
    # Initialize Spark context
    #spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

    #sc = spark.sparkContext

    # Read the file into an RDD
    # Each line in the file becomes one record in the RDD
    lines = sc.textFile(filename)

    # Process each line in the RDD
    data = lines.map(lambda line: line.split(','))\
                .map(lambda elem: ([float(x) for x in elem[:-1]], int(elem[-1])))

    return data

In [3]:
#Testing of the read function
filename = 'data/botnet_tot_syn_l.csv'
RDD_Xy = readFile(filename)

# Printing first 5 rows of the RDD
for row in RDD_Xy.take(5):
    print(row)

([9.012784269851089, 1672.9999766891833, 21.99998846107087, 0.9999997452701503, 61.99988768910407, 69.99980788311223, 13.000000232993186, 2.9999999785934133, 199.00000192395984, 2468369573.0148935, 2468372549.224571], 1)
([3599.9990884099398, 48206.57583515873, 13362.999847123127, 1.0000019205086303, 262.99924185046257, 82.99988759205826, 13.999945225127506, 5.000000032871429, 216.99999612378573, 1539044199.5873349, 2468368394.7593513], 0)
([0.0006999386757797765, 0.0009472844685660675, 53.000260213743786, 2.0000004196928174, 646242903.4069201, 82.99988759205826, 12.999999992936175, 2.999999985175615, 216.99999612378573, 2468369544.907854, 2503250078.1888514], 0)
([3599.9990884099398, 1718.7743566535646, 63377.68872854763, 1.0000019205086303, 926682.197305462, 543.5568914038613, 12.999999992936175, 5.000000032871429, 186.99999400676145, 405749204.8178947, 1125424493.3033822], 0)
([3471.3792185025277, 48024.1859162382, 0.00035088937602267833, 507008.9762772573, 262.99924185046257, 1000.

In [47]:
'''
def compute_stats(RDD):
    # Calculate sum and sum of squares for each feature
    sums = RDD.map(lambda x: x[0]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))])
    return sums
print(compute_stats(rdd))
'''

'\ndef compute_stats(RDD):\n    # Calculate sum and sum of squares for each feature\n    sums = RDD.map(lambda x: x[0]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))])\n    return sums\nprint(compute_stats(rdd))\n'

In [4]:
def compute_mean_stds(RDD):
    # Calculate sum and sum of squares for each feature
    sums = RDD.map(lambda x: x[0]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))]) 
    sum_of_squares = RDD.map(lambda x: [xi ** 2 for xi in x[0]]).reduce(lambda a, b: [a[i] + b[i] for i in range(len(a))])
    count = RDD.count()

    # Compute mean and standard deviation for each feature
    means = [sums[i] / count for i in range(len(sums))]
    stds = [(sum_of_squares[i] / count - (means[i]) ** 2) ** 0.5 for i in range(len(means))]
    
    return means, stds



def normalize(RDD_Xy):
    # Compute means and standard deviations
    means, stds = compute_mean_stds(RDD_Xy)

    # Broadcast the means and standard deviations
    bc_means = sc.broadcast(means)
    bc_stds = sc.broadcast(stds)

    # Normalize each feature
    def normalize_features(features, means, stds):
        return [(features[i] - means[i]) / stds[i] if stds[i] != 0 else 0 for i in range(len(features))]
        #return [(features[i] - means[i]) / stds[i] for i in range(len(features))]

    normalized_RDD = RDD_Xy.map(lambda x: (normalize_features(x[0], bc_means.value, bc_stds.value), x[1]))

    return normalized_RDD



In [5]:
#Testing of the normalize function

normalized_RDD = normalize(RDD_Xy).cache()

# Print first 5 normalized rows
for row in normalized_RDD.take(5):
    print(row)


([-0.792409774778866, -0.8130937083307339, -0.4224507642218906, -0.4664697491193844, -0.5223929554497563, -0.35631956924945285, 0.7370102960327225, 0.5283496333423401, 0.8271779941619601, 0.4731661571170488, 0.15895172357621995], 1)
([1.443720404002351, 1.1163610034798641, 0.39127563577819524, -0.4664697491031832, -0.5223920961870128, -0.35631927462193325, 0.9263319360863808, 1.480120748267978, 1.0264183320400917, -0.8286954523619166, 0.15894853159524694], 0)
([-0.7980216574316705, -0.8824624555329105, -0.4205599209717913, -0.46646230111358483, 2.240264662197529, -0.35631927462193325, 0.7370102505822355, 0.5283496364747148, 1.0264183320400917, 0.4731661177428138, 0.185749013466657], 0)
([1.443720404002351, -0.8111957326975887, 3.441892018491165, -0.4664697491031832, -0.5184316978805608, -0.35630883678072567, 0.7370102505822355, 1.480120748267978, 0.6943509718073647, -2.416291452741655, -0.8728693208325027], 0)
([1.3636279869526706, 1.108798440707592, -0.42379261911074273, 3.30972605187

In [6]:
import numpy as np
from pyspark import SparkContext

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def linear_combination(features, weights, bias):
    return sum(weights[j] * features[j] for j in range(len(weights))) + bias

def compute_gradients_and_cost(record, weights, bias):
    xi, yi = record
    y_hat = sigmoid(linear_combination(xi, weights, bias))
    error = y_hat - yi
    dw = np.zeros(len(weights))
    for j in range(len(weights)):
        dw[j] = error * xi[j]
    db = error
    cost_contribution = -yi * np.log(y_hat) - (1 - yi) * np.log(1 - y_hat)
    return (dw, db, cost_contribution)

def train(RDD_Xy, iterations, learning_rate, lambda_reg):
    np.random.seed(0) 
    feature_count = len(RDD_Xy.first()[0])
    w = np.random.rand(feature_count)
    b = np.random.rand()
    m = RDD_Xy.count()

    for n in range(iterations):
        bc_w = sc.broadcast(w)
        bc_b = sc.broadcast(b)

        gradients_and_cost = RDD_Xy.map(lambda x: compute_gradients_and_cost(x, bc_w.value, bc_b.value))
        dw, db, total_cost = gradients_and_cost.reduce(lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2]))

        reg_cost = (lambda_reg / (2 * feature_count)) * np.sum(np.square(w))
        cost = (total_cost / m) + reg_cost

        w -= learning_rate * ((dw / m) + (lambda_reg/m) * w)
        b -= learning_rate * db / m

        bc_w.unpersist()
        bc_b.unpersist()

    return w, b


In [7]:
def predict(w, b, x):
    z = 0
    for j in range(len(w)):
        z += w[j] * x[j]

    z+= b
    # Compute the sigmoid of z
    y_hat = sigmoid(z)

    if y_hat > 0.5:
        return 1
    else: 
        return 0




In [8]:
def accuracy(w, b, RDD_Xy):
    
    results = RDD_Xy.map(lambda r: 1 if predict(w, b, r[0]) == r[1] else 0)
    results = results.reduce( lambda a, b: a + b)

    accuracy = results / RDD_Xy.count()
    return accuracy




In [41]:
'''

# Additional function to assign fold indices
def assign_fold_index(RDD_Xy, num_folds):
    num_samples = RDD_Xy.count()
    fold_sizes = (num_samples // num_folds) * np.ones(num_folds, dtype=int)
    fold_sizes[:num_samples % num_folds] += 1
    fold_indices = np.hstack([np.full(fold_size, fold_index) for fold_index, fold_size in enumerate(fold_sizes)])
    np.random.shuffle(fold_indices)
    return RDD_Xy.zipWithIndex().map(lambda x: (fold_indices[x[1]], x[0]))

# Cross-validation function
def cross_validate(RDD_Xy, num_folds, iterations, learning_rate, lambda_reg):
    # Assign each row to a fold
    indexed_RDD = assign_fold_index(RDD_Xy, num_folds)

    # Broadcast the indexed RDD
    bc_indexed_RDD = sc.broadcast(indexed_RDD.collect())

    # Initialize array to hold accuracy for each fold
    fold_accuracies = []

    # Perform cross-validation
    for fold in range(num_folds):
        train_RDD = sc.parallelize([x[1] for x in bc_indexed_RDD.value if x[0] != fold])
        test_RDD = sc.parallelize([x[1] for x in bc_indexed_RDD.value if x[0] == fold])
        
        # Train the model on the training set
        w, b = train(train_RDD, iterations, learning_rate, lambda_reg)

        # Evaluate the model on the validation set
        fold_acc = accuracy(w, b, test_RDD)
        fold_accuracies.append(fold_acc)
        print(f"Fold {fold+1}/{num_folds} - Accuracy: {fold_acc}")

    # Calculate average accuracy across all folds
    avg_acc = sum(fold_accuracies) / num_folds
    return avg_acc

'''

'\n\n# Additional function to assign fold indices\ndef assign_fold_index(RDD_Xy, num_folds):\n    num_samples = RDD_Xy.count()\n    fold_sizes = (num_samples // num_folds) * np.ones(num_folds, dtype=int)\n    fold_sizes[:num_samples % num_folds] += 1\n    fold_indices = np.hstack([np.full(fold_size, fold_index) for fold_index, fold_size in enumerate(fold_sizes)])\n    np.random.shuffle(fold_indices)\n    return RDD_Xy.zipWithIndex().map(lambda x: (fold_indices[x[1]], x[0]))\n\n# Cross-validation function\ndef cross_validate(RDD_Xy, num_folds, iterations, learning_rate, lambda_reg):\n    # Assign each row to a fold\n    indexed_RDD = assign_fold_index(RDD_Xy, num_folds)\n\n    # Broadcast the indexed RDD\n    bc_indexed_RDD = sc.broadcast(indexed_RDD.collect())\n\n    # Initialize array to hold accuracy for each fold\n    fold_accuracies = []\n\n    # Perform cross-validation\n    for fold in range(num_folds):\n        train_RDD = sc.parallelize([x[1] for x in bc_indexed_RDD.value if x[

In [42]:
'''

def assign_uniform_blocks(RDD, num_blocks=10):

    #here it is used a list because the integer are not global variable, so if we modify the value in the function, the
    #change will not be seen by the next call of the function.
    block_counter = [0]

    def assign_block(record):
        
        block_num = (block_counter[0] % num_blocks) + 1
        block_counter[0] += 1
        #(block_num,) -> the ',' is used to explicit that block_num is not an integer, but a single tuple of an integer.
        #record + (block_num,) -> is the concatenation between the tuple of record with the new one.
        return record + (block_num,)

    RDD_with_blocks = RDD.map(assign_block)

    return RDD_with_blocks

RDD_Xy_with_blocks = assign_uniform_blocks(normalized_RDD)

for row in RDD_Xy_with_blocks.take(10):
    print(row)

'''

([-0.792409774778866, -0.8130937083307339, -0.4224507642218906, -0.4664697491193844, -0.5223929554497563, -0.35631956924945285, 0.7370102960327225, 0.5283496333423401, 0.8271779941619601, 0.4731661571170488, 0.15895172357621995], 1, 1)
([1.443720404002351, 1.1163610034798641, 0.39127563577819524, -0.4664697491031832, -0.5223920961870128, -0.35631927462193325, 0.9263319360863808, 1.480120748267978, 1.0264183320400917, -0.8286954523619166, 0.15894853159524694], 0, 2)
([-0.7980216574316705, -0.8824624555329105, -0.4205599209717913, -0.46646230111358483, 2.240264662197529, -0.35631927462193325, 0.7370102505822355, 0.5283496364747148, 1.0264183320400917, 0.4731661177428138, 0.185749013466657], 0, 3)
([1.443720404002351, -0.8111957326975887, 3.441892018491165, -0.4664697491031832, -0.5184316978805608, -0.35630883678072567, 0.7370102505822355, 1.480120748267978, 0.6943509718073647, -2.416291452741655, -0.8728693208325027], 0, 4)
([1.3636279869526706, 1.108798440707592, -0.42379261911074273, 3

In [28]:
import numpy as np

def get_uniform_number(min=1, max=10):
    # Generate a float number from uniform distribution
    float_number = np.random.uniform(min, max + 0.9999)

    # Round the float number to the nearest integer, ensuring it's between 1 and 10, inclusive
    int_number = int(np.clip(float_number, 1, 10))

    return int_number


def assign_uniform_blocks(RDD):

    def assign_block(record):
        block_num = get_uniform_number(0, 10)
        return record + (block_num,)

    RDD_with_blocks = RDD.map(assign_block)

    return RDD_with_blocks



RDD_Xy_with_blocks = assign_uniform_blocks(normalized_RDD)

for row in RDD_Xy_with_blocks.take(10):
    print(row)

([-0.792409774778866, -0.8130937083307339, -0.4224507642218906, -0.4664697491193844, -0.5223929554497563, -0.35631956924945285, 0.7370102960327225, 0.5283496333423401, 0.8271779941619601, 0.4731661571170488, 0.15895172357621995], 1, 2)
([1.443720404002351, 1.1163610034798641, 0.39127563577819524, -0.4664697491031832, -0.5223920961870128, -0.35631927462193325, 0.9263319360863808, 1.480120748267978, 1.0264183320400917, -0.8286954523619166, 0.15894853159524694], 0, 6)
([-0.7980216574316705, -0.8824624555329105, -0.4205599209717913, -0.46646230111358483, 2.240264662197529, -0.35631927462193325, 0.7370102505822355, 0.5283496364747148, 1.0264183320400917, 0.4731661177428138, 0.185749013466657], 0, 3)
([1.443720404002351, -0.8111957326975887, 3.441892018491165, -0.4664697491031832, -0.5184316978805608, -0.35630883678072567, 0.7370102505822355, 1.480120748267978, 0.6943509718073647, -2.416291452741655, -0.8728693208325027], 0, 9)
([1.3636279869526706, 1.108798440707592, -0.42379261911074273, 3

In [30]:
def cross_validate(RDD_Xy_with_blocks, num_blocks=10, iterations=10, learning_rate=1.5, lambda_reg=0):
    total_accuracy = 0

    for test_block in range(1, num_blocks + 1):
        
        training_RDD = RDD_Xy_with_blocks.flatMap(lambda x: [(x[0], x[1])] if x[-1] != test_block else [])
        test_RDD = RDD_Xy_with_blocks.flatMap(lambda x: [(x[0], x[1])] if x[-1] == test_block else [])

        w, b = train(training_RDD, iterations, learning_rate, lambda_reg)

        current_accuracy = accuracy(w, b, test_RDD)
        print(f"Accuracy for block {test_block} as test set: {current_accuracy}")

        total_accuracy += current_accuracy

    average_accuracy = total_accuracy / num_blocks
    return average_accuracy

average_accuracy = cross_validate(RDD_Xy_with_blocks)
print("Average Cross-Validation Accuracy:", average_accuracy)


Accuracy for block 1 as test set: 0.9293428448380578


Accuracy for block 2 as test set: 0.9275197938575723


Accuracy for block 3 as test set: 0.9333451202263083


Accuracy for block 4 as test set: 0.9255704395108474


Accuracy for block 5 as test set: 0.9263267772772222


Accuracy for block 6 as test set: 0.9319403445389258


Accuracy for block 7 as test set: 0.920392465364276


Accuracy for block 8 as test set: 0.9355328441457001


Accuracy for block 9 as test set: 0.9267234294355502


Accuracy for block 10 as test set: 0.9372205766521339
Average Cross-Validation Accuracy: 0.9293914635846594
